<a href="https://colab.research.google.com/github/TranQuocViet26701/word2vec/blob/main/BTL_WordEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisite Mathematical Concepts

Before diving into the Word2Vec models, we need to understand three fundamental mathematical concepts that form the backbone of these algorithms:

1. **Conditional Probability** - Understanding how the probability of one event changes given information about another event
2. **Maximum Likelihood Estimation (MLE)** - A method for finding the best parameters that explain our observed data
3. **Softmax Function** - A way to convert raw scores into probabilities

Let's explore each concept in detail with step-by-step explanations and examples.

## 3.b Conditional Probability

### What is Conditional Probability?

**Conditional probability** answers the question: *"What is the probability of event A happening, given that we already know event B has happened?"*

In everyday language, we use conditional probability all the time:
- "What's the chance of rain **given that** the sky is cloudy?"
- "What's the probability a student passes the exam **given that** they studied for 10 hours?"

In Word2Vec, we ask similar questions:
- "What's the probability of seeing the word 'cat' **given that** the center word is 'pet'?"
- "What's the probability of the center word being 'coffee' **given that** the surrounding words are 'I', 'drink', 'every', 'morning'?"

### The Formal Definition

The conditional probability of event $A$ given event $B$ is written as $P(A \mid B)$ and is defined as:

$$P(A \mid B) = \frac{P(A \cap B)}{P(B)}$$

Where:
- $P(A \mid B)$ is read as "the probability of A given B"
- $P(A \cap B)$ is the probability that **both** A and B happen (the intersection)
- $P(B)$ is the probability that B happens
- **Important**: This formula only makes sense when $P(B) > 0$ (we can't condition on an impossible event)

### Intuitive Understanding

Think of conditional probability as **narrowing down your sample space**.

Imagine you have a bag with 10 balls:
- 4 red balls
- 6 blue balls
- Among the red balls: 3 are large, 1 is small
- Among the blue balls: 2 are large, 4 are small

**Question 1**: What's the probability of picking a large ball?
- Total large balls = 3 + 2 = 5
- Total balls = 10
- $P(\text{Large}) = \frac{5}{10} = 0.5$

**Question 2**: What's the probability of picking a large ball, **given that** we already know it's red?
- Now we only consider the red balls (our sample space is reduced!)
- Red balls = 4
- Large red balls = 3
- $P(\text{Large} \mid \text{Red}) = \frac{3}{4} = 0.75$

Notice how knowing the ball is red **changed** the probability from 0.5 to 0.75!

### Step-by-Step Verification Using the Formula

Let's verify our answer using the formal formula $P(A \mid B) = \frac{P(A \cap B)}{P(B)}$

**Given**:
- Event $A$ = picking a large ball
- Event $B$ = picking a red ball

**Step 1**: Calculate $P(B)$ - the probability of picking a red ball
$$P(\text{Red}) = \frac{\text{Number of red balls}}{\text{Total balls}} = \frac{4}{10} = 0.4$$

**Step 2**: Calculate $P(A \cap B)$ - the probability of picking a ball that is BOTH large AND red
$$P(\text{Large} \cap \text{Red}) = \frac{\text{Number of large red balls}}{\text{Total balls}} = \frac{3}{10} = 0.3$$

**Step 3**: Apply the conditional probability formula
$$P(\text{Large} \mid \text{Red}) = \frac{P(\text{Large} \cap \text{Red})}{P(\text{Red})} = \frac{0.3}{0.4} = \frac{3}{4} = 0.75$$

This matches our intuitive answer of $0.75$.

### The Chain Rule of Probability

From the conditional probability formula, we can derive an extremely useful result called the **Chain Rule**:

Starting from:
$$P(A \mid B) = \frac{P(A \cap B)}{P(B)}$$

We can rearrange by multiplying both sides by $P(B)$:
$$P(A \cap B) = P(A \mid B) \times P(B)$$

This tells us: **The probability of A AND B happening equals the probability of B happening times the probability of A given B**.

**Example**: What's the probability of drawing two aces in a row from a deck of cards (without replacement)?

Let:
- $A$ = second card is an ace
- $B$ = first card is an ace

**Step 1**: $P(B) = P(\text{First card is ace}) = \frac{4}{52} = \frac{1}{13}$

**Step 2**: $P(A \mid B) = P(\text{Second card is ace} \mid \text{First card was ace})$
- After drawing one ace, there are 3 aces left and 51 cards total
- $P(A \mid B) = \frac{3}{51} = \frac{1}{17}$

**Step 3**: Apply chain rule
$$P(A \cap B) = P(A \mid B) \times P(B) = \frac{1}{17} \times \frac{1}{13} = \frac{1}{221} \approx 0.0045$$

### Extension to Multiple Events

The chain rule extends to multiple events:
$$P(A, B, C) = P(A) \times P(B \mid A) \times P(C \mid A, B)$$

In general, for events $X_1, X_2, \ldots, X_n$:
$$P(X_1, X_2, \ldots, X_n) = P(X_1) \times P(X_2 \mid X_1) \times P(X_3 \mid X_1, X_2) \times \cdots \times P(X_n \mid X_1, \ldots, X_{n-1})$$

This is crucial for modeling sequences of words in NLP!

### Connection to Word2Vec

In Word2Vec, conditional probability is the core concept that drives how word embeddings are learned.

**Skip-Gram Model** asks: Given a center word, what is the probability of each context word?
$$P(w_{\text{context}} \mid w_{\text{center}})$$

For example, given the sentence "The **cat** sat on the mat" with "cat" as the center word and window size 2:
- $P(\text{"The"} \mid \text{"cat"}) = ?$
- $P(\text{"sat"} \mid \text{"cat"}) = ?$
- $P(\text{"on"} \mid \text{"cat"}) = ?$

**CBOW Model** asks: Given the context words, what is the probability of the center word?
$$P(w_{\text{center}} \mid w_{\text{context}_1}, w_{\text{context}_2}, \ldots)$$

For example: $P(\text{"cat"} \mid \text{"The"}, \text{"sat"}, \text{"on"}) = ?$

**Key Insight**: Word2Vec learns word vectors such that words appearing in similar contexts have similar vectors. The conditional probability framework allows us to:
1. Formulate this as a prediction problem
2. Use maximum likelihood (next section) to find the best word vectors
3. Use softmax (later section) to convert similarity scores into probabilities

---

## 3.c Maximum Likelihood Estimation (MLE)

### What is Maximum Likelihood Estimation?

**Maximum Likelihood Estimation (MLE)** is a method for finding the best parameters of a statistical model. The idea is simple:

> *"Find the parameter values that make the observed data most probable."*

In other words, among all possible parameter values, we choose the ones that would most likely produce the data we actually observed.

### Likelihood vs Probability: Understanding the Difference

This distinction is subtle but crucial:

**Probability**: Given fixed parameters, what's the chance of observing certain data?
- "If a coin has 50% heads probability, what's the chance of getting 7 heads in 10 flips?"
- We fix $\theta = 0.5$ and ask about $P(\text{data})$

**Likelihood**: Given observed data, how plausible are different parameter values?
- "I observed 7 heads in 10 flips. Is the coin's heads probability more likely 0.5 or 0.7?"
- We fix the data and ask about different values of $\theta$

**Mathematical notation**:
- Probability: $P(\text{Data} \mid \theta)$ - probability of data given fixed parameter $\theta$
- Likelihood: $\mathcal{L}(\theta \mid \text{Data})$ - likelihood of parameter $\theta$ given fixed data

Numerically, they are the same value! But conceptually:
- In probability, $\theta$ is fixed and Data varies
- In likelihood, Data is fixed and $\theta$ varies

### The MLE Formula

The Maximum Likelihood Estimate is the parameter value $\hat{\theta}$ that maximizes the likelihood:

$$\hat{\theta}_{MLE} = \underset{\theta}{\arg\max} \; \mathcal{L}(\theta \mid \text{Data}) = \underset{\theta}{\arg\max} \; P(\text{Data} \mid \theta)$$

Where:
- $\hat{\theta}_{MLE}$ is the MLE estimate (the "best" parameter value)
- $\underset{\theta}{\arg\max}$ means "find the value of $\theta$ that maximizes..."
- $\mathcal{L}(\theta \mid \text{Data})$ is the likelihood function

### Step-by-Step Example: The Coin Flip Problem

**Problem**: You flip a coin 10 times and observe 7 heads and 3 tails. What is the most likely value for the coin's probability of heads?

**Setup**:
- Let $\theta$ = probability of heads (what we want to find)
- Observed data: 7 heads (H) and 3 tails (T) in 10 flips
- Each flip is independent

**Step 1: Write the likelihood function**

For a single coin flip:
- $P(\text{Heads} \mid \theta) = \theta$
- $P(\text{Tails} \mid \theta) = 1 - \theta$

Since all flips are independent, we multiply the probabilities:
$$\mathcal{L}(\theta) = P(\text{7 heads, 3 tails} \mid \theta) = \theta^7 \times (1-\theta)^3$$

**Step 2: Try some values to build intuition**

Let's calculate the likelihood for different values of $\theta$:

| $\theta$ | $\theta^7$ | $(1-\theta)^3$ | $\mathcal{L}(\theta) = \theta^7 \times (1-\theta)^3$ |
|----------|------------|----------------|-----------------------------------------------------|
| 0.5 | 0.0078 | 0.125 | 0.000977 |
| 0.6 | 0.0280 | 0.064 | 0.001792 |
| 0.7 | 0.0824 | 0.027 | 0.002224 |
| 0.8 | 0.2097 | 0.008 | 0.001678 |

Notice that $\theta = 0.7$ gives the highest likelihood! This matches our intuition: 7 heads out of 10 suggests the probability should be around 70%.

**Step 3: Find the exact maximum using calculus**

To find the exact value of $\theta$ that maximizes the likelihood, we take the derivative and set it to zero.

Our likelihood function is:
$$\mathcal{L}(\theta) = \theta^7 \times (1-\theta)^3$$

**Step 3a**: Apply the product rule to find $\frac{d\mathcal{L}}{d\theta}$

Product rule: $\frac{d}{dx}[f(x) \cdot g(x)] = f'(x) \cdot g(x) + f(x) \cdot g'(x)$

Let $f(\theta) = \theta^7$ and $g(\theta) = (1-\theta)^3$

Then:
- $f'(\theta) = 7\theta^6$ (power rule)
- $g'(\theta) = 3(1-\theta)^2 \times (-1) = -3(1-\theta)^2$ (chain rule)

Therefore:
$$\frac{d\mathcal{L}}{d\theta} = 7\theta^6 \cdot (1-\theta)^3 + \theta^7 \cdot (-3)(1-\theta)^2$$

**Step 3b**: Simplify

Factor out common terms $\theta^6(1-\theta)^2$:
$$\frac{d\mathcal{L}}{d\theta} = \theta^6(1-\theta)^2 \left[ 7(1-\theta) - 3\theta \right]$$

Simplify the bracket:
$$= \theta^6(1-\theta)^2 \left[ 7 - 7\theta - 3\theta \right]$$
$$= \theta^6(1-\theta)^2 \left[ 7 - 10\theta \right]$$

**Step 3c**: Set derivative to zero and solve

$$\theta^6(1-\theta)^2(7 - 10\theta) = 0$$

This equals zero when:
- $\theta = 0$ (trivial solution - coin never shows heads)
- $\theta = 1$ (trivial solution - coin always shows heads)
- $7 - 10\theta = 0 \Rightarrow \theta = 0.7$ (**this is our MLE!**)

**Conclusion**: $\hat{\theta}_{MLE} = 0.7 = \frac{7}{10} = \frac{\text{number of heads}}{\text{total flips}}$

This is a beautiful result! The MLE for a coin's bias is simply the observed proportion of heads.

### Why Use Log-Likelihood?

In practice, we almost always work with the **log-likelihood** instead of the likelihood directly:

$$\ell(\theta) = \log \mathcal{L}(\theta)$$

**Why take the logarithm?**

**Reason 1: Numerical Stability**

When we have many data points, the likelihood becomes a product of many small probabilities:
$$\mathcal{L}(\theta) = P(x_1|\theta) \times P(x_2|\theta) \times \cdots \times P(x_n|\theta)$$

For example, with 1000 data points each with probability 0.01:
$$\mathcal{L}(\theta) = 0.01^{1000} = 10^{-2000}$$

This number is so small that computers cannot represent it (underflow)!

**Reason 2: Products Become Sums**

The logarithm transforms products into sums:
$$\log(a \times b) = \log(a) + \log(b)$$

So:
$$\ell(\theta) = \log \mathcal{L}(\theta) = \log P(x_1|\theta) + \log P(x_2|\theta) + \cdots + \log P(x_n|\theta)$$

$$= \sum_{i=1}^{n} \log P(x_i|\theta)$$

Sums are much easier to compute and differentiate than products!

**Reason 3: Same Maximum**

Since $\log$ is a monotonically increasing function:
- If $a > b$, then $\log(a) > \log(b)$

Therefore, maximizing $\mathcal{L}(\theta)$ is equivalent to maximizing $\log \mathcal{L}(\theta)$:
$$\underset{\theta}{\arg\max} \; \mathcal{L}(\theta) = \underset{\theta}{\arg\max} \; \log \mathcal{L}(\theta)$$

### Coin Flip Example with Log-Likelihood

Let's redo our coin flip example using log-likelihood.

**Original likelihood**:
$$\mathcal{L}(\theta) = \theta^7 \times (1-\theta)^3$$

**Step 1: Take the logarithm**
$$\ell(\theta) = \log(\theta^7 \times (1-\theta)^3)$$

Using $\log(a \times b) = \log(a) + \log(b)$:
$$\ell(\theta) = \log(\theta^7) + \log((1-\theta)^3)$$

Using $\log(a^n) = n \times \log(a)$:
$$\ell(\theta) = 7\log(\theta) + 3\log(1-\theta)$$

**Step 2: Find the derivative**
$$\frac{d\ell}{d\theta} = 7 \times \frac{1}{\theta} + 3 \times \frac{-1}{1-\theta}$$
$$= \frac{7}{\theta} - \frac{3}{1-\theta}$$

**Step 3: Set to zero and solve**
$$\frac{7}{\theta} - \frac{3}{1-\theta} = 0$$

Multiply both sides by $\theta(1-\theta)$:
$$7(1-\theta) - 3\theta = 0$$
$$7 - 7\theta - 3\theta = 0$$
$$7 = 10\theta$$
$$\theta = 0.7$$

**Same answer**, but the calculation was simpler! No product rule needed.

### From MLE to Minimizing Loss: The Word2Vec Connection

In Word2Vec, we use MLE to find the best word vectors. Here's how it works:

**The Setup**:
- We have a text corpus with words $w_1, w_2, \ldots, w_T$
- We want to find word vectors (parameters $\theta$) that maximize the probability of observing these word sequences

**For Skip-Gram**: Given center word $w_c$, predict context words $w_o$

The likelihood of the entire corpus is:
$$\mathcal{L}(\theta) = \prod_{t=1}^{T} \prod_{-m \leq j \leq m, j \neq 0} P(w^{(t+j)} \mid w^{(t)}; \theta)$$

**Taking the log**:
$$\ell(\theta) = \sum_{t=1}^{T} \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}; \theta)$$

**The Key Transformation: Maximization → Minimization**

In machine learning, we typically **minimize** a loss function rather than maximize likelihood. The conversion is simple:

$$\text{Maximize } \ell(\theta) \quad \Longleftrightarrow \quad \text{Minimize } -\ell(\theta)$$

So the **loss function** for Skip-Gram is:
$$\mathcal{L}_{loss} = -\ell(\theta) = -\sum_{t=1}^{T} \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}; \theta)$$

This is called **Negative Log-Likelihood (NLL)** loss!

**Why minimize negative log-likelihood?**
1. Convention: ML frameworks are designed to minimize, not maximize
2. Positive values: Loss is always non-negative (since $\log P \leq 0$ for $P \leq 1$)
3. Interpretation: Lower loss = higher likelihood = better model

---

## 3.d The Softmax Function

### What is Softmax?

The **softmax function** converts a vector of arbitrary real numbers into a **probability distribution**. It takes a vector of "scores" (which can be any real numbers, positive or negative) and outputs a vector of probabilities that:
1. Are all positive (between 0 and 1)
2. Sum to exactly 1

This makes softmax perfect for multi-class classification where we need to predict the probability of each class.

### The Softmax Formula

Given a vector of scores $\mathbf{z} = (z_1, z_2, \ldots, z_K)$ where $K$ is the number of classes, the softmax function outputs:

$$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

Or for the entire vector:
$$\text{softmax}(\mathbf{z}) = \left( \frac{e^{z_1}}{\sum_{j=1}^{K} e^{z_j}}, \frac{e^{z_2}}{\sum_{j=1}^{K} e^{z_j}}, \ldots, \frac{e^{z_K}}{\sum_{j=1}^{K} e^{z_j}} \right)$$

Where:
- $e \approx 2.71828$ is Euler's number (base of natural logarithm)
- $e^{z_i}$ is the exponential of $z_i$
- The denominator $\sum_{j=1}^{K} e^{z_j}$ is a **normalization constant** that ensures the outputs sum to 1

### Step-by-Step Numerical Example

Let's compute softmax for the vector $\mathbf{z} = (2, 1, 0.5)$

**Step 1: Compute the exponential of each element**

| $i$ | $z_i$ | $e^{z_i}$ | Calculation |
|-----|-------|-----------|-------------|
| 1 | 2 | $e^2 = 7.389$ | $2.71828^2 = 7.389$ |
| 2 | 1 | $e^1 = 2.718$ | $2.71828^1 = 2.718$ |
| 3 | 0.5 | $e^{0.5} = 1.649$ | $2.71828^{0.5} = 1.649$ |

**Step 2: Calculate the sum of all exponentials (denominator)**

$$\sum_{j=1}^{3} e^{z_j} = e^2 + e^1 + e^{0.5} = 7.389 + 2.718 + 1.649 = 11.756$$

**Step 3: Divide each exponential by the sum**

| $i$ | $e^{z_i}$ | $\text{softmax}(z_i) = \frac{e^{z_i}}{11.756}$ | Probability |
|-----|-----------|------------------------------------------------|-------------|
| 1 | 7.389 | $\frac{7.389}{11.756}$ | $0.628$ (62.8%) |
| 2 | 2.718 | $\frac{2.718}{11.756}$ | $0.231$ (23.1%) |
| 3 | 1.649 | $\frac{1.649}{11.756}$ | $0.140$ (14.0%) |

**Step 4: Verify probabilities sum to 1**

$$0.628 + 0.231 + 0.140 = 0.999 \approx 1.0 \; \checkmark$$

(The small error is due to rounding)

**Interpretation**: The input with the highest score (2) gets the highest probability (62.8%), but the other classes still get non-zero probabilities.

### Why Use the Exponential Function?

You might wonder: why $e^{z_i}$ instead of just $z_i$? There are several important reasons:

**1. Always Positive**

The exponential function $e^x$ is always positive for any real number $x$:
- $e^{-100} = 3.7 \times 10^{-44}$ (very small, but positive!)
- $e^{0} = 1$
- $e^{100} = 2.7 \times 10^{43}$ (very large)

This ensures all probabilities are positive, which is required!

**2. Amplifies Differences**

The exponential function amplifies differences between scores:

| $z_1$ | $z_2$ | Difference | $e^{z_1}$ | $e^{z_2}$ | Ratio $\frac{e^{z_1}}{e^{z_2}}$ |
|-------|-------|------------|-----------|-----------|-------------------------------|
| 3 | 2 | 1 | 20.09 | 7.39 | 2.72 |
| 5 | 2 | 3 | 148.4 | 7.39 | 20.09 |

A score difference of 1 leads to a probability ratio of about 2.72 ($\approx e$).

**3. Differentiable**

The exponential function has a nice derivative: $\frac{d}{dx}e^x = e^x$

This makes it easy to compute gradients for training neural networks with gradient descent.

**4. Preserves Ranking**

If $z_1 > z_2$, then $e^{z_1} > e^{z_2}$, so $\text{softmax}(z_1) > \text{softmax}(z_2)$

The relative ordering is preserved.

### Connection to Word2Vec

In Word2Vec, the softmax function is crucial for converting similarity scores between word vectors into probabilities.

**Skip-Gram Model**: Given a center word $w_c$ with vector $\mathbf{v}_c$, we want to compute the probability of each word $w_o$ being a context word.

**Step 1**: Compute the "score" for each word using dot product

The dot product $\mathbf{u}_o^\top \mathbf{v}_c$ measures how similar the context word vector $\mathbf{u}_o$ is to the center word vector $\mathbf{v}_c$:
- High dot product → words are similar → high score
- Low dot product → words are different → low score

**Step 2**: Convert scores to probabilities using softmax

$$P(w_o \mid w_c) = \text{softmax}(\mathbf{u}_o^\top \mathbf{v}_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}$$

Where:
- $\mathbf{u}_o$ is the "context" vector for word $w_o$
- $\mathbf{v}_c$ is the "center" vector for word $w_c$
- $\mathcal{V}$ is the entire vocabulary
- The denominator sums over ALL words in the vocabulary

**Example Intuition**:

Consider the center word "coffee" and three potential context words:

| Context Word | Dot Product Score | After Softmax |
|--------------|-------------------|---------------|
| "drink" | 5.0 (high similarity) | 0.72 |
| "morning" | 3.5 (moderate similarity) | 0.24 |
| "airplane" | 0.5 (low similarity) | 0.04 |

The softmax converts these raw similarity scores into a valid probability distribution!

**The Computational Challenge**:

Notice that the denominator $\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)$ sums over the **entire vocabulary**. 

If the vocabulary has 3 million words, this means:
- 3 million dot products
- 3 million exponentials
- 3 million additions

**For every single training example!** This is why approximate methods like Negative Sampling (covered earlier in this notebook) are used in practice.

---

# Self-Supervised word2vec

Self-Supervised word2vec
1. Giới thiệu về cách embed word thành vector hiệu quả hơn one-hot, sử dụng xác suất, học tự giám sát.
2. The Skip-Gram Model: Diễn giải lý thuyết mô hình, cách xây dựng công thức P từ softmax, max likelihood, giải thích hàm loss cho pha training (tại sao lại biến đổi ra hàm L đó từ max likelihood)
3. The Continuous Bag of Words (CBOW) Model: Ngược lại với Skip-Gram là với input context words tính xác suất sinh ra center word. Trình bày: Giải thích công thức xác suất điều kiện của mô hình, max likelihood, công thức pha training.

## Self-Supervised Word to Vector Method

To beyond the limitations of One-Hot method, there was an approach based on Self-Supervised learning with two architectures: The Skip-Gram model and the Continuous Bag-of-Words (CBOW) model [(Mikolov et al., 2013)](https://arxiv.org/pdf/1301.3781).

Following this method, they do not treat words as atomic units – there is no notion of similarity between words. Instead, both CBOW and skip-gram learn distributed representations by leveraging the local context in which words appear. The CBOW model predicts a target word based on its surrounding words, effectively aggregating contextual information to infer meaning. Conversely, the skip-gram model predicts the surrounding context words given a single target word, aiming to learn word vectors that are informative enough to generate their typical neighbors in text. Together, these two approaches capture semantic and syntactic regularities by exploiting patterns that naturally occur in large corpora.

This approach is grounded in the use of conditional probability, where models learn word representations by estimating the likelihood of a target word given its context or vice versa. Through these probability-based predictions, the embeddings capture meaningful semantic relationships directly from unlabeled text in the corpus - the reason why this method is self-supervised.

### Skip-Gram Model

### The Continuous Bag of Words Model

Why do we need to use approximate training instead of the full softmax?

## Fully Softmax
In general, according to (15.1.4)
the log conditional probability
involving any pair of the center word $w_c$ and
the context word $w_o$ is

$$\log P(w_o \mid w_c) =\mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right).$$

where $\mathcal{|V|}$ = vocabulary size so complexity for full softmax cost per training pair

$$O(\mathcal{|V|})$$


For a typical Word2Vec with a vocabulary size of $|V| = 3{,}000{,}000$:

- $\approx 3{,}000{,}000$ dot products per update  
- $\approx 3{,}000{,}000$ gradient updates

## Negative Sampling

Replace the full softmax probability with a binary logistic classifier (Nagetive Sampling):

- For each real pair (context, target):

  - Predict $D = 1$

- For k noise words sampled from a unigram distribution:

  - Predict $D = 0$

**Cost per step:**

$$O(\mathcal{k})$$

where typically $k=5$ to $10$ $15$.

So instead of 3M updates, you only do ~10 updates. Massive speedup $\approx 500,000$x faster than full softmax.

## Hierarchical Softmax

Use a Huffman tree to reduce softmax from:

$$O(\mathcal{|V|})$$

to:

$$O(\log_2{|V|})$$

For a vocabulary size of 3M words:

$$O(\log_2{|3,000,000|}) \approx 22$$

So each update uses $\log$ ~20 nodes instead of millions.

## Negative Sampling in Skip-Gram Model (SG)

Negative sampling transform the multi-classification into binary classification.

Given:
- The center word $w_c$
- The context word $w_o$

The probability model will be:

$$P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c)$$

Based on (15.1.5), given:
- The text sequence of length $T$
- The word at time step $t$ is $w^{(t)}$
- The context window size be $m$

The joint probability will be:

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)})$$

The formula only considers those events that involve positive examples ($D = 1$). The joint probability is maximized to $1$ only if $v_c^\top v_w \to +\infty$. In other words, all the word vectors are equal to infinity. We are expecting that adding negative examples ($D = 0$) will make more sense.

Given:
- $S$ is the event that a context word $w_o$ comes from the context window of a center word $w_c$
- With predefined distribution $P(w)$ sample $K$ noise words, $N_k$ is the event that a noise word $w_k$ ($k = 1, ..., K$)

So these events involving both the positive example and negative examples are $\{S, N_1, ..., N_K \}$.

Negative sampling rewrites the conditional probability:

$$P(w^{(t+j)} \mid w^{(t)})$$
$$= P_S \prod_K P_k $$
$$= P(D=1\mid w^{(t)}, w^{(t+j)}) \prod_{k=1,\,w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k) $$

Given:
- $i_t$ is index of a word $w^{(t)}$ at time step $t$
- $h_k$ is index of a noise word $w_k$

The logarithmic loss

$$-\log P(w^{(t+j)} \mid w^{(t)})$$
$$= -\log P(D=1\mid w^{(t)}, w^{(t+j)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)$$

because of classification binary so $P(D=0\mid w^{(t)}, w_k) = 1 - P(D=1\mid w^{(t)}, w_k)$, we have:

$$= -\log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)\right)$$

with $\sigma(x) + \sigma(-x) = 1$, We can infer:

$$= -\log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)$$




## Negative Sampling in Conitnuous Bag-of-Word Model (CBOW)

Given:
- The center word $w_c$
- The context vector $\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}} \right)/(2m)$

The probability model will be:

$$P(D=1\mid w_c, w_{o_1},..., w_{o_{2m}}) = \sigma(\mathbf{u}_o^\top \bar{\mathbf{v}}_o)$$

Based on (15.1.12), given:
- The text sequence of length $T$
- The word at time step $t$ is $w^{(t)}$
- The context window size be $m$

The joint probability will be:

$$ \prod_{t=1}^{T}  P(D=1 \mid w^{(t)},  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)})$$

Given:
- $S$ is the event that a context word $w_o$ comes from the context window of a center word $w_c$
- With predefined distribution $P(w)$ sample $K$ noise words, $N_k$ is the event that a noise word $w_k$ ($k = 1, ..., K$)

Add negative examples

$$P(w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}) \mid w^{(t)})$$
$$= P_S \prod_K P_k $$
$$= P(D=1\mid w^{(t)},  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}) \prod_{k=1,\,w_k \sim P(w)}^K P(D=0\mid w_k, w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}) $$


The logarithmic loss:

$$-\log P(w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}) \mid w^{(t)})$$

$$= -\log P(D=1\mid w^{(t)},  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w_k, w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)})$$


$$= -\log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)\right)$$

with $\sigma(x) + \sigma(-x) = 1$, We can infer (todo: cleaning):

$$= -\log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \bar{\mathbf{v}}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)$$
